In [1]:
!pip install gmdh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.3/875.3 kB 13.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import plotly
import plotly.express as px
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from gmdh import Combi, Multi, Mia, Ria, split_data, PolynomialType, CriterionType
from gmdh import SequentialCriterion, Solver
from gmdh import Criterion
from gmdh import ParallelCriterion
from gmdh import SequentialCriterion, Solver
from sklearn.preprocessing import LabelEncoder

In [3]:
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
train_data.drop('Id', axis=1,inplace=True)

In [5]:
Q1 = train_data['SalePrice'].quantile(0.25)
Q3 = train_data['SalePrice'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

train_data = train_data[(train_data['SalePrice'] >= lower_bound) & (train_data['SalePrice'] <= upper_bound)]

In [6]:
print(len(train_data))
check_data = train_data[1200:]
train_data = train_data[:1200]

1399


In [7]:
train_data_nums = train_data.select_dtypes(include = ['float64', 'int64'])
test_data_nums = test_data.select_dtypes(include = ['float64', 'int64'])

In [8]:
X, y = train_data_nums.drop('SalePrice', axis=1), train_data_nums['SalePrice']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=155)

In [9]:
X_train = X_train.fillna(X_train.mean())
X_valid = X_valid.fillna(X_valid.mean())

In [10]:
par_criterion_1 = ParallelCriterion(criterion_type=CriterionType.SYM_STABILITY, 
                                  second_criterion_type=CriterionType.SYM_REGULARITY,
                                  solver=Solver.ACCURATE,
                                  alpha=0.85)

par_criterion_2 = ParallelCriterion(criterion_type=CriterionType.STABILITY, 
                                  second_criterion_type=CriterionType.SYM_ABSOLUTE_NOISE_IMMUNITY,
                                  solver=Solver.ACCURATE,
                                  alpha=0.85)

In [11]:
%%time
model_gmdh = Ria() #Ria, Multi, Mia, Combi


model_gmdh.fit(np.array(X_train), np.array(y_train), par_criterion_1, polynomial_type=PolynomialType.LINEAR_COV,k_best=5,n_jobs=-1,p_average=1) #LINEAR_COV, QUADRATIC, LINEAR
y_predicted = model_gmdh.predict(X_valid)

print(mean_squared_error(y_valid, y_predicted))

321314200.4166316
CPU times: user 520 ms, sys: 9.38 ms, total: 530 ms
Wall time: 145 ms


In [12]:
test_data_nums = test_data_nums.fillna(test_data_nums.mean())   
test_predict = model_gmdh.predict(test_data_nums.drop('Id',axis=1))

In [13]:
output_1 = pd.DataFrame({'Id':test_data.Id,
                       'SalePrice':test_predict})

In [14]:
X, y = train_data.drop('SalePrice', axis=1), train_data['SalePrice']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=155)

In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900 entries, 386 to 996
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     900 non-null    int64  
 1   MSZoning       900 non-null    object 
 2   LotFrontage    734 non-null    float64
 3   LotArea        900 non-null    int64  
 4   Street         900 non-null    object 
 5   Alley          55 non-null     object 
 6   LotShape       900 non-null    object 
 7   LandContour    900 non-null    object 
 8   Utilities      900 non-null    object 
 9   LotConfig      900 non-null    object 
 10  LandSlope      900 non-null    object 
 11  Neighborhood   900 non-null    object 
 12  Condition1     900 non-null    object 
 13  Condition2     900 non-null    object 
 14  BldgType       900 non-null    object 
 15  HouseStyle     900 non-null    object 
 16  OverallQual    900 non-null    int64  
 17  OverallCond    900 non-null    int64  
 18  YearBuilt    

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {
    'n_estimators': [2000],  # Можно изменить диапазон
    'max_depth': [3],           # Можно изменить диапазон
    'learning_rate': [0.01], # Можно изменить диапазон
    'subsample': [0.3],     # Можно изменить диапазон
}


# param_grid = {
#     'n_estimators': [1800, 1900, 2000, 2100, 2200, 2300],  # Можно изменить диапазон
#     'max_depth': [3],           # Можно изменить диапазон
#     'learning_rate': [0.009, 0.008, 0.01, 0.02, 0.03], # Можно изменить диапазон
#     'subsample': [0.25, 0.3, 0.35],     # Можно изменить диапазон
# }

# Создаем модель XGBoost
model = xgb.XGBRegressor(eval_metric='logloss', tree_method="hist", objective='reg:squarederror', enable_categorical=True)

# Создаем объект GridSearchCV для поиска по сетке
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Разделяем данные на обучающий и тестовый наборы
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = [
    "MSSubClass", "MSZoning", "Street", "Alley", "LotShape", "LandContour",
    "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1",
    "Condition2", "HouseStyle", "OverallQual", "OverallCond", "RoofStyle",
    "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "ExterQual",
    "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure",
    "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", "Electrical",
    "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish",
    "GarageQual", "GarageCond", "PavedDrive", "PoolQC", "Fence", "MiscFeature",
    "SaleType", "SaleCondition", "BldgType", "CentralAir"
]

for feature in categorical_features:  
    X_train[feature] = X_train[feature].astype("category")
    X_valid[feature] = X_valid[feature].astype("category")
    

# Запускаем поиск по сетке
grid_search.fit(X_train, y_train)

# Получаем лучшие параметры
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

# Создаем модель с лучшими параметрами
best_model = xgb.XGBRegressor(**best_params, eval_metric='logloss', tree_method="hist", objective='reg:squarederror', enable_categorical=True)

# Обучаем модель с лучшими параметрами
best_model.fit(X_train, y_train)

# Делаем предсказания
y_pred = best_model.predict(X_valid)
y_train_pred = best_model.predict(X_train)

# Выводим результаты
print(f"Среднеквадратичная ошибка (MSE) на обучающем наборе: {mean_squared_error(y_train, y_train_pred)}")
print(f"Среднеквадратичная ошибка (MSE) на тестовом наборе: {mean_squared_error(y_valid, y_pred)}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Лучшие параметры: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 2000, 'subsample': 0.3}
Среднеквадратичная ошибка (MSE) на обучающем наборе: 71407987.2594346
Среднеквадратичная ошибка (MSE) на тестовом наборе: 753841652.444031


In [17]:
for feature in categorical_features: 
    try:
        test_data[feature] = test_data[feature].astype("category")
    except:
        pass
    
test_predict = best_model.predict(test_data.drop('Id',axis=1))

In [18]:
output_2 = pd.DataFrame({'Id':test_data.Id,
                       'SalePrice':test_predict})

In [19]:
ans = 0.25*output_1['SalePrice'] + 0.75*output_2['SalePrice']

In [20]:
output = pd.DataFrame({'Id':test_data.Id,
                       'SalePrice':ans})

output.to_csv('submission.csv',index=False)

In [21]:
output

,Id,SalePrice
0,1461,134188.460678
1,1462,163353.586292
2,1463,187038.283406
3,1464,197948.948326
4,1465,185775.577287
...,...,...
1454,2915,79905.806100
1455,2916,79847.155959
1456,2917,176179.646104
1457,2918,127520.023786
